# Tensorflow コンテナのカスタマイズ
## 概要
### 注意
Readme.mdにもあるように、ノートブックインスタンスにAmazonSageMakerFullAccessとAmazonEC2ContainerRegistryFullAccessのロールを付与してください。レジストリの作成やpushを行うためです。

### 概要

Tensorflowなどの各種Deep Learningフレームワークに対応したコンテナイメージを、ユーザ自身が作成できるようにするため、各種Dockerファイルが公開されています。  
https://github.com/aws/sagemaker-tensorflow-container  
https://github.com/aws/sagemaker-mxnet-container  
https://github.com/aws/sagemaker-chainer-container  
https://github.com/aws/sagemaker-pytorch-container  
このノートブックでは、Tensorflow1.10のコンテナイメージをビルドし、ECRにpushして、SageMakerで利用するまでの流れを説明します。

## コンテナイメージの作成
Dockerファイルをダウンロードして編集し、最後にビルドします。

### (1) Dockerファイルのcloneと編集

`git clone`を利用してdockerファイルをcloneします。ファイルは`sagemaker-tensorflow-container/docker/1.10.0/`以下に、`Dockerfile.gpu`と`Dockerfile.cpu`に保存されます。gpuとついているのはGPUを利用するためのDockerfile, cpuとついているのはCPUを利用するためのDockerfileとなります。

In [ ]:
!git clone https://github.com/aws/sagemaker-tensorflow-container.git

`git clone`が終わるとそれぞれのファイルを編集します。例えば、Approximate Nearest Neighborの[Annoy](https://github.com/spotify/annoy)をコンテナイメージに入れたい場合は、Dockerfileの最後に以下を追記します。ただし現状はpipインストール可能なライブラリのためにBYOCをする必要はありません。Estimatorにrequirements.txtを渡すと、pipインストールされます。詳しくはhttps://sagemaker.readthedocs.io/en/latest/sagemaker.tensorflow.html
```
RUN pip install annoy
```

### (2-1) GPU向けPython3系Tensorflowのコンテナイメージ

こちらはTensorflow1.10.0を利用する場合のビルトになります。基本的な流れは

https://github.com/aws/sagemaker-tensorflow-container

に説明されており、以下のスクリプトは、その説明をひとまとめにしたものです。

In [ ]:
!cd sagemaker-tensorflow-container && python setup.py sdist
!cd sagemaker-tensorflow-container/ && cp  dist/sagemaker_tensorflow_container-*.tar.gz docker/1.10.0/
!cd sagemaker-tensorflow-container/docker/1.10.0/final/py2 && \
wget https://files.pythonhosted.org/packages/64/ca/830b7cedb073ae264d215d51bd18d7cff7a2a47e39d79f6fa23edae17bb2/tensorflow_gpu-1.10.0-cp36-cp36m-manylinux1_x86_64.whl &&  \
docker build -t preprod-tensorflow:1.10.0-gpu-py3 --build-arg py_version=3 --build-arg framework_installable=tensorflow_gpu-1.10.0-cp36-cp36m-manylinux1_x86_64.whl -f Dockerfile.gpu .

INFO: From Compiling external/org_tensorflow/tensorflow/core/kernels/cwise_op_gpu_digamma.cu.cc:
external/org_tensorflow/tensorflow/core/kernels/cwise_ops.h(190): warning: __host__ annotation on a defaulted function("scalar_left") is ignored

external/org_tensorflow/tensorflow/core/kernels/cwise_ops.h(190): warning: __device__ annotation on a defaulted function("scalar_left") is ignored

external/org_tensorflow/tensorflow/core/kernels/cwise_ops.h(220): warning: __host__ annotation on a defaulted function("scalar_right") is ignored

external/org_tensorflow/tensorflow/core/kernels/cwise_ops.h(220): warning: __device__ annotation on a defaulted function("scalar_right") is ignored

external/org_tensorflow/tensorflow/core/kernels/cwise_ops.h(190): warning: __host__ annotation on a defaulted function("scalar_left") is ignored

external/org_tensorflow/tensorflow/core/kernels/cwise_ops.h(190): warning: __device__ annotation on a defaulted function("scalar_left") is ignored

external/org_tensor

### (2-2) CPU向けPython3系Tensorflowのコンテナイメージ

In [ ]:
!cd sagemaker-tensorflow-container && python setup.py sdist
!cd sagemaker-tensorflow-container/ && cp  dist/sagemaker_tensorflow_container-*.tar.gz docker/1.10.0/
!cd sagemaker-tensorflow-container/docker/1.10.0/final/py2 && \
wget https://files.pythonhosted.org/packages/ee/e6/a6d371306c23c2b01cd2cb38909673d17ddd388d9e4b3c0f6602bfd972c8/tensorflow-1.10.0-cp36-cp36m-manylinux1_x86_64.whl &&  \
docker build -t preprod-tensorflow:1.10.0-cpu-py3 --build-arg py_version=3 --build-arg framework_installable=tensorflow-1.10.0-cp36-cp36m-manylinux1_x86_64.whl -f Dockerfile.cpu .

### (3) Dockerイメージの確認
REPOSITORYが`preprod-tensorflow`、TAGが`1.10.0-cpu-py3`と`1.10.0-gpu-py3`の2つができていることを確認します。前者がcpu、後者がgpu用のコンテナイメージです。
    

In [ ]:
!docker images

### (4) DockerイメージをECRにpush
コンテナイメージの内容を確認したら、レポジトリの名前やタグなどをメモしておき、それらをECRにアップロードします。大まかな流れは：
1. boto3でECRにレポジトリを作成(メモしたレポジトリの名前を入れる）
1. ECRにログイン
1. docker tag と docker pushでそれぞれのコンテナをpush


In [ ]:
import boto3

# Create repository with AWS SDK boto3
rep_name = 'preprod-tensorflow'
client = boto3.client('ecr')
response = client.create_repository(repositoryName=rep_name)
repo_url = response['repository']['repositoryUri']

# Specifying names
gpu_tag = '1.10.0-gpu-py3'
cpu_tag = '1.10.0-cpu-py3'
gpu_name = rep_name + ':' + gpu_tag
gpu_repo_name = repo_url + ':' + gpu_tag
cpu_name = rep_name + ':' + cpu_tag
cpu_repo_name = repo_url + ':' + cpu_tag


# Login to ECR
!$(aws ecr get-login --region ap-northeast-1 --no-include-email)

#push cpu image to ECR
!docker tag  $cpu_name $cpu_repo_name
!docker push $cpu_repo_name

#push cpu image to ECR
!docker tag  $gpu_name $gpu_repo_name
!docker push $gpu_repo_name

## ここからは通常のSageMakerの使い方になります

使用しているのはこちらです。コメントもこちらをご覧ください。  
https://github.com/awslabs/amazon-sagemaker-examples/tree/master/sagemaker-python-sdk/tensorflow_distributed_mnist

### 環境セットアップ

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

### MNISTデータセットのダウンロード

In [ ]:
import utils
from tensorflow.contrib.learn.python.learn.datasets import mnist
import tensorflow as tf

data_sets = mnist.read_data_sets('data', dtype=tf.uint8, reshape=False, validation_size=5000)

utils.convert_to(data_sets.train, 'train', 'data')
utils.convert_to(data_sets.validation, 'validation', 'data')
utils.convert_to(data_sets.test, 'test', 'data')

### データのアップロード

In [ ]:
inputs = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-mnist')

### 学習の実行

まず今回利用するコンテナイメージを確認します。cpuを使うならcpu_repo_name, gpuを使うならgpu_repo_nameを使います。

In [ ]:
print(cpu_repo_name)

CPUなら`image_name = cpu_repo_name`としてTensorflowのEstimatorを作り、学習・デプロイを行います。ここでは学習のみ行います。

In [ ]:
from sagemaker.tensorflow import TensorFlow

mnist_estimator = TensorFlow(entry_point='mnist.py',
                             role=role,
                             image_name= cpu_repo_name,
                             framework_version='1.10.0',
                             training_steps=100, 
                             evaluation_steps=10,
                             train_instance_count=1,
                             train_instance_type='ml.c4.xlarge')

mnist_estimator.fit(inputs)